In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
xls = pd.ExcelFile(f'./var/GEO402.Homework1.data.2020.xlsx.xlsx')
names = xls.sheet_names

In [36]:
for i in range(11):
    tmax = pd.read_excel(xls, names[i*3]).replace([-99], np.nan)
    tmin = pd.read_excel(xls, names[i*3+1]).replace([-99], np.nan)
    prcp = pd.read_excel(xls, names[i*3+2]).replace([-99], np.nan)

    tmean = ((tmax + tmin) / 2.)
    tm = (tmean.values[:30, 1:-1] -32.) * 5. / 9. 
    tm = np.nanmean(tm, axis=0)

    pm = np.nanmean(prcp.values[:30, 1:-1] * 25.4, axis=0)

    X = np.arange(1, 13, 1)
    fig, ax = plt.subplots(1, 1, sharex=True, figsize=(8, 6))
    ax.bar(X, pm, facecolor='purple', edgecolor='k', width=.5)
    ax.set_ylim(0, 250)
    ax.set_ylabel('Precipitation (mm)', fontweight='bold', color='k', fontsize='large')
    ax.set_yticks(np.arange(0, 260, 50))
    ax.set_yticklabels(np.arange(0, 260, 50), color='k', fontsize='large')
    ax.set_xlabel('Month', fontweight='bold', color='k', fontsize='large')
    ax.set_xticks(np.arange(1, 13, 1))
    ax.set_xticklabels(np.arange(1, 13, 1), color='k', fontsize='large')
    ax.legend(['Precipitation'], frameon=False, bbox_to_anchor=(0.5, .5, .5, 0.5), fontsize='large')

    axx = ax.twinx()
    axx.plot(X, tm, '-*')
    axx.set_ylim(-15, 35)
    axx.set_ylabel('Temperature (C)', fontweight='bold', color='k', fontsize='large')
    axx.set_yticks(np.arange(-15, 40, 5))
    axx.set_yticklabels(np.arange(-15, 40, 5), color='k', fontsize='large')
    axx.legend(['Temperature'], frameon=False, bbox_to_anchor=(0.5, .45, .5, 0.5), fontsize='large')
    axx.set_title(f'Monthly Mean Precipitation and Temperature at {names[i*3][:-3]} 1981-2010', fontweight='bold', fontsize='large')

    plt.savefig(f'./plot/GEO402/{names[i*3][:-3]}.png', bbox_inches='tight')
    plt.close()

In [61]:
climatecategory = []
prcpstd = []
Trange = []
FrostFreeMon = []
cv = []
for i in range(11):
    tmax = pd.read_excel(xls, names[i*3]).replace([-99], np.nan)
    tmin = pd.read_excel(xls, names[i*3+1]).replace([-99], np.nan)
    prcp = pd.read_excel(xls, names[i*3+2]).replace([-99], np.nan)

    tmean = ((tmax + tmin) / 2.)
    tm = (tmean.values[:30, 1:-1] -32.) * 5. / 9. 
    MAT = tm.mean(axis=1).mean()
    tm = np.nanmean(tm, axis=0)
    Thot = np.max(tm)
    Tcold = np.min(tm)
    Tmon10 = np.where(tm>10, 1, 0).sum()
    Trange.append(Thot-Tcold)
    FrostFreeMon.append(np.where(tm>0, 1, 0))
    
    pm = prcp.values[:30, 1:-1] * 25.4
    MAP = pm.sum(axis=1).mean()
    pm = np.nanmean(pm, axis=0)
    prcpstd.append(np.std(pm))
    cv.append(np.std(pm)/pm.mean())
    
    Pdry = np.min(pm)
    Psdry = np.min(pm[3:9])
    Pwdry = np.min(pm[np.r_[0:3, 9:12]])
    Pswet = np.max(pm[3:9])
    Pwwet = np.max(pm[np.r_[0:3, 9:12]])
    if pm[np.r_[0:3, 9:12]].sum() / pm.sum() > .7:
        Pthreshold = 2. * MAT
    elif pm[3:9].sum() / pm.sum() > .7:
        Pthreshold = 2. * MAT + 28.
    else:
        Pthreshold = 2. * MAT + 14.
        
    o1 = FirstOrder(MAP, Pthreshold, Tcold, Thot)
    o2 = SecondOrder(o1, Pdry, MAP, Pthreshold, Psdry, Pwwet, Pwdry, Pswet, Thot)
    o3 = ThridOrder(o1, MAT, Thot, Tmon10, Tcold)
    
    climatecategory.append(o1+o2+o3)

In [4]:
def FirstOrder(MAP, Pthreshold, Tcold, Thot):
    if MAP < 10*Pthreshold:
        return 'B'
    elif Tcold >= 18:
        return 'A'
    elif (Thot > 10) and (Tcold > 0) and (Tcold < 18):
        return 'C'
    elif (Thot > 10) and (Tcold <= 0):
        return 'D'
    elif Thot < 10:
        return 'E'

In [7]:
def SecondOrder(o1, Pdry, MAP, Pthreshold, Psdry, Pwwet, Pwdry, Pswet, Thot):
    if o1 == 'A':
        if Pdry >= 60.:
            return 'f'
        elif Pdry >= (100. - MAP/25.):
            return 'm'
        elif Pdry < (100. - MAP/25.):
            return 'w'
    if o1 == 'B':
        if MAP < 5*Pthreshold:
            return 'W'
        elif MAP >= 5*Pthreshold:
            return 'S'
    if (o1 == 'C') or (o1 == 'D'):
        if (Psdry < 40) and (Psdry < Pwwet/3.):
            return 's'
        elif Pwdry < Pswet/10.:
            return 'w'
        else:
            return 'f'
    if o1 == 'E':
        if Thot > 0:
            return 'T'
        elif Thot <= 0:
            return 'F'

In [30]:
def ThridOrder(o1, MAT, Thot, Tmon10, Tcold):
    if o1 == 'B':
        if MAT >= 18:
            return 'h'
        elif MAT < 18:
            return 'k'
    elif o1 == 'C':
        if Thot >= 22:
            return 'a'
        elif Tmon10 >= 4:
            return 'b'
        elif (Tmon10 >= 1) and (Tmon10 < 4):
            return 'c'
    elif o1 == 'D':
        if Thot >= 22:
            return 'a'
        elif Tmon10 >= 4:
            return 'b'
        elif Tcold < -38:
            return 'd'
        else:
            return 'c'
    else:
        return ''

In [47]:
# data = [MAT, MAP, Pthreshold, Tcold, Thot, Pdry, Psdry, Pwwet, Pwdry, Pswet, Tmon10]
# name = ['MAT', 'MAP', 'Pthreshold', 'Tcold', 'Thot', 'Pdry', 'Psdry', 'Pwwet', 'Pwdry', 'Pswet', 'Tmon10']
# pd.DataFrame({name[i]:[f'{data[i]:.2f}'] for i in range(11)})
# o1+o2+o3

In [62]:
pd.DataFrame({names[i*3][:-3]:[f'{cv[i]:.2f}'] for i in range(11)})

,Atlanta,Boston,Denver,Fargo,Houston,KansasCity,Lansing,LA,Miami,Seattle,Tucson
0,0.13,0.09,0.49,0.54,0.22,0.46,0.26,1.01,0.57,0.59,0.68


In [60]:
df = pd.DataFrame({names[i*3][:-3]:FrostFreeMon[i] for i in range(11)})
df.rename(index={i:(i+1) for i in range(12)})

,Atlanta,Boston,Denver,Fargo,Houston,KansasCity,Lansing,LA,Miami,Seattle,Tucson
1,1,0,0,0,1,0,0,1,1,1,1
2,1,1,1,0,1,1,0,1,1,1,1
3,1,1,1,0,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,1
10,1,1,1,1,1,1,1,1,1,1,1
